In [4]:
import numpy as np
import pandas as pd
import glob

from backend.app import num_random_images

In [5]:
path = './resources/unsplash-dataset/'
documents = ['photos', 'keywords', 'collections', 'conversions', 'colors']
datasets = {}

for doc in documents:
  files = glob.glob(path + doc + ".tsv*")

  subsets = []
  for filename in files:
    df = pd.read_csv(filename, sep='\t', header=0)
    subsets.append(df)

  datasets[doc] = pd.concat(subsets, axis=0, ignore_index=True)

In [7]:
datasets['photos'].columns

Index(['photo_id', 'photo_url', 'photo_image_url', 'photo_submitted_at',
       'photo_featured', 'photo_width', 'photo_height', 'photo_aspect_ratio',
       'photo_description', 'photographer_username', 'photographer_first_name',
       'photographer_last_name', 'exif_camera_make', 'exif_camera_model',
       'exif_iso', 'exif_aperture_value', 'exif_focal_length',
       'exif_exposure_time', 'photo_location_name', 'photo_location_latitude',
       'photo_location_longitude', 'photo_location_country',
       'photo_location_city', 'stats_views', 'stats_downloads',
       'ai_description', 'ai_primary_landmark_name',
       'ai_primary_landmark_latitude', 'ai_primary_landmark_longitude',
       'ai_primary_landmark_confidence', 'blur_hash'],
      dtype='object')

In [29]:
start_index = 5639
end_index = 10000

In [30]:
import os
import requests

# resize all images to max 2000 dimension, pillow thumbnail
from PIL import Image

image_urls = datasets['photos']['photo_image_url'].tolist()
i = start_index
# Watch out, won't be in sync with the index of the image as previously some images failed to download. Will cause url_index to need to be later than i
# Seems to be a problem with url_index 5639 only
url_index = start_index
while i < end_index:
    try:
        response = requests.get(image_urls[url_index], stream=True)
        response.raise_for_status()  # Check for errors

        # Save the image
        with open(f"resources/random_images/image_{i}.jpg", "wb") as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)

    except Exception as e:
        print(f"Failed to download {image_urls[i]}: {e}")
        url_index += 1
        continue
    image_path = f'./resources/random_images/image_{i}.jpg'
    image = Image.open(image_path)
    image = image.convert('RGB')
    image.thumbnail((2000, 2000))
    image.save(f'./resources/random_images/image_{i}.jpg')
    i += 1
    url_index += 1


Failed to download https://images.unsplash.com-grass-sun.jpg: HTTPSConnectionPool(host='images.unsplash.com-grass-sun.jpg', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x17dfdfc50>: Failed to resolve 'images.unsplash.com-grass-sun.jpg' ([Errno 8] nodename nor servname provided, or not known)"))


/Users/duyhuynh/Documents/Anaglyph AI/.venv/lib/python3.13/site-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (99991727 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/Users/duyhuynh/Documents/Anaglyph AI/.venv/lib/python3.13/site-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (143040000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/Users/duyhuynh/Documents/Anaglyph AI/.venv/lib/python3.13/site-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (91177320 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/Users/duyhuynh/Documents/Anaglyph AI/.venv/lib/python3.13/site-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (99996120 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/Users/duyhuynh/Documents/Anaglyph AI/.venv/lib

In [32]:
from depth_map_generator import depth_map_generator
import cv2
import numpy as np

# Generate depth maps in greyscale jpeg
if not os.path.exists('./resources/random_images_depth_maps_greyscale'):
    os.makedirs('./resources/random_images_depth_maps_greyscale')
for i in range(end_index):
    image_path = f'./resources/random_images/image_{i}.jpg'
    image = cv2.imread(image_path)
    depth_map = depth_map_generator.generate_depth_map(image)
    depth_map_greyscale = (depth_map * 255).astype(np.uint8)
    cv2.imwrite(f'./resources/random_images_depth_maps_greyscale/depth_map_greyscale_{i}.jpg', depth_map_greyscale)



In [11]:
# Generate greyscale depth maps to use jpeg compression

for i in range(num_random_images):
    depth_map_path = f'./resources/random_images_depth_maps/depth_map_{i}.npy'
    depth_map = np.load(depth_map_path)
    depth_map_greyscale = (depth_map * 255).astype(np.uint8)
    cv2.imwrite(f'./resources/random_images_depth_maps_greyscale/depth_map_greyscale_{i}.jpg', depth_map_greyscale)